## LOAD EXAMPLE DATA FROM RETAIL TABLE

In [21]:
import sys
import os

# Get the parent directory of the current notebook (project_root)
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))

# Add the project root directory to the Python path
sys.path.append(project_root)



In [2]:
import numpy as np 
import pandas as pd 
import chardet



# Load the CSV data into a DataFrame
csv_file = 'Online Retail Data Set.csv'  # Replace with the actual path to your CSV file

with open(csv_file, 'rb') as f:
    result = chardet.detect(f.read())
    
data = pd.read_csv(csv_file, encoding=result['encoding'])


# Display basic information about the DataFrame
print("DataFrame Info:")
print(data.info())

# Display the first few rows of the DataFrame
print("\nFirst few rows of the DataFrame:")
print(data.head())


DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB
None

First few rows of the DataFrame:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BO

In [17]:
import pandas as pd

# Create or load your original DataFrame
# df = ...

# Get the unique countries from the "country" column
unique_countries = data["Country"].unique()

# Randomly select one user from each unique country
selected_rows = []
for country in unique_countries:
    country_df = data[data["Country"] == country]
    if not country_df.empty:
        selected_row = country_df.sample(n=1, random_state=42)
        selected_rows.append(selected_row)

# Concatenate the selected rows to create the mini version of the DataFrame
mini_df = pd.concat(selected_rows)


In [24]:
df = data.sample(n=20000)

In [22]:
from main.models import ModelClass

txt = ModelClass()

In [106]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Embeddings
213256,555526,23199,JUMBO BAG APPLES,100,05-06-2011 11:42,1.79,14364.0,United Kingdom,"[[0.019880796, 0.06821113, -0.03838431, 0.0200..."
409879,572077,21485,RETROSPOT HEART HOT WATER BOTTLE,6,20-10-2011 14:09,4.95,15807.0,United Kingdom,"[[0.030709248, 0.042524725, -0.016905049, 0.00..."
445700,574856,23284,DOORMAT KEEP CALM AND COME IN,3,07-11-2011 12:06,15.79,NaN,United Kingdom,"[[0.043934163, 0.040514313, -0.036823496, 0.03..."
369301,569026,47566,PARTY BUNTING,4,30-09-2011 10:18,4.95,16101.0,United Kingdom,"[[0.020661157, 0.047775645, -0.014194804, 0.02..."
77336,542719,21986,PACK OF 12 PINK POLKADOT TISSUES,4,31-01-2011 14:18,0.29,17619.0,United Kingdom,"[[0.005416592, 0.002898387, -0.019814478, 0.01..."
...,...,...,...,...,...,...,...,...,...
525658,580635,22767,TRIPLE PHOTO FRAME CORNICE,2,05-12-2011 12:30,9.95,14649.0,United Kingdom,"[[0.008097997, 0.029931773, -0.041610282, 0.02..."
220867,556237,21001,ROSE DU SUD WASHBAG,1,09-06-2011 15:34,4.13,NaN,United Kingdom,"[[0.015630372, 0.050903413, -0.016166765, 0.01..."
155012,549958,22208,WOOD STAMP SET THANK YOU,2,13-04-2011 13:47,1.65,17589.0,United Kingdom,"[[0.016032154, 0.05473344, -0.031026268, 0.016..."
76800,542659,22090,PAPER BUNTING RETROSPOT,6,31-01-2011 12:09,2.95,15738.0,United Kingdom,"[[0.03224307, 0.021167545, -0.027737578, 0.020..."


In [138]:
def generate_inventory(your_df, tag_column: str = "tags", separator: str = ","):
    """
    Helper function that creates a master inventory for all the products in historical DataFrame.

    Args:
        df (DataFrame): The historical DataFrame containing product information.
        tag_column (str, optional): The name of the column containing product tags. Default is 'tags'.
        separator (str, optional): The separator used to split tags. Default is a comma.

    Returns:
        dict: A dictionary representing the product inventory with tag counts.
    """

    inv = {}

    for tags in your_df[tag_column]:
        if type(tags) != float: 
            tag_list = tags.split(separator)
        else: continue 
        for tag in tag_list:
            tag = tag.strip()  # Remove leading/trailing spaces
            if tag and not tag.isnumeric():  # Skip numeric tags
                inv[tag] = inv.get(tag, 0) + 1  # Increment tag count

    return inv


def create_profiles(
    your_df, user_idex_column: str = "user_id", tag_column: str = "tags"
):
    """
    Helper function that creates a unique row for just one user, including all purchases or likes in all rows of historical purchases.

    Args:
        your_df (DataFrame): The historical DataFrame containing user information.
        user_idex_column (str): The column name representing the user ID. Default is 'user_id'.
        tag_column (str,optional) -- default:'tags' -- : The name of the column containing product tags.

    Returns:
        DataFrame: A DataFrame containing user profile information.
    """
    
    # Group by user_id and concatenate the products into a single row
    user_profiles = (
        your_df.groupby(user_idex_column)[tag_column]
        .apply(lambda x: ",".join(x))
        .reset_index()
    )
    user_profiles.rename(
        columns={user_idex_column: "user_id", tag_column: "likes"}, inplace=True
    )

    return user_profiles

In [139]:
df_1= create_profiles(df,'CustomerID','Description')
df_2= generate_inventory(df,'Description',' ')

AttributeError: module 'pandas' has no attribute 'isnumber'

In [110]:
df_1 

,user_id,likes
0,12347.0,WOODLAND CHARLOTTE BAG REGENCY TEAPOT ROSES E...
1,12349.0,GLASS SONGBIRD STORAGE JAR SET OF 6 SPICE TIN...
2,12352.0,DELUXE SEWING KIT CERAMIC CAKE DESIGN SPOTTED...
3,12354.0,CIRCUS PARADE BABY GIFT SET
4,12356.0,SET OF 3 HEART COOKIE CUTTERS SET/10 BLUE POLK...
...,...,...
3085,18269.0,HOT WATER BOTTLE BABUSHKA
3086,18272.0,RED POLKADOT COFFEE MUG DANISH ROSE DECORATIV...
3087,18278.0,HOT WATER BOTTLE TEA AND SYMPATHY
3088,18282.0,PAPER BUNTING VINTAGE PAISLEY


In [61]:
import pandas as pd
import numpy as np

# Create or load your Series object
series = df['Embeddings'] 

# Check if each element in the Series is an array with dtype float32
is_valid = series.apply(lambda x: isinstance(x, np.ndarray) and x.dtype == np.float32)

# Filter out rows that don't meet the criteria
filtered_series = series[is_valid]

print(filtered_series)


213256    [[0.019880796, 0.06821113, -0.03838431, 0.0200...
409879    [[0.030709248, 0.042524725, -0.016905049, 0.00...
445700    [[0.043934163, 0.040514313, -0.036823496, 0.03...
369301    [[0.020661157, 0.047775645, -0.014194804, 0.02...
77336     [[0.005416592, 0.002898387, -0.019814478, 0.01...
                                ...                        
525658    [[0.008097997, 0.029931773, -0.041610282, 0.02...
220867    [[0.015630372, 0.050903413, -0.016166765, 0.01...
155012    [[0.016032154, 0.05473344, -0.031026268, 0.016...
76800     [[0.03224307, 0.021167545, -0.027737578, 0.020...
154450    [[-0.024462353, 0.032406785, -0.017173741, 0.0...
Name: Embeddings, Length: 20000, dtype: object


In [65]:
df.Embeddings

213256    [[0.019880796, 0.06821113, -0.03838431, 0.0200...
409879    [[0.030709248, 0.042524725, -0.016905049, 0.00...
445700    [[0.043934163, 0.040514313, -0.036823496, 0.03...
369301    [[0.020661157, 0.047775645, -0.014194804, 0.02...
77336     [[0.005416592, 0.002898387, -0.019814478, 0.01...
                                ...                        
525658    [[0.008097997, 0.029931773, -0.041610282, 0.02...
220867    [[0.015630372, 0.050903413, -0.016166765, 0.01...
155012    [[0.016032154, 0.05473344, -0.031026268, 0.016...
76800     [[0.03224307, 0.021167545, -0.027737578, 0.020...
154450    [[-0.024462353, 0.032406785, -0.017173741, 0.0...
Name: Embeddings, Length: 20000, dtype: object

In [84]:
df1 = pd.read_excel("data.xlsx")
df1

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Embeddings
0,213256,555526,23199,JUMBO BAG APPLES,100,05-06-2011 11:42,1.79,14364.0,United Kingdom,[[ 1.98807959e-02 6.82111308e-02 -3.83843109e...
1,409879,572077,21485,RETROSPOT HEART HOT WATER BOTTLE,6,20-10-2011 14:09,4.95,15807.0,United Kingdom,[[ 3.07092480e-02 4.25247252e-02 -1.69050489e...
2,445700,574856,23284,DOORMAT KEEP CALM AND COME IN,3,07-11-2011 12:06,15.79,NaN,United Kingdom,[[ 4.39341627e-02 4.05143127e-02 -3.68234962e...
3,369301,569026,47566,PARTY BUNTING,4,30-09-2011 10:18,4.95,16101.0,United Kingdom,[[ 2.06611566e-02 4.77756448e-02 -1.41948042e...
4,77336,542719,21986,PACK OF 12 PINK POLKADOT TISSUES,4,31-01-2011 14:18,0.29,17619.0,United Kingdom,[[ 5.41659212e-03 2.89838691e-03 -1.98144782e...
...,...,...,...,...,...,...,...,...,...,...
19995,525658,580635,22767,TRIPLE PHOTO FRAME CORNICE,2,05-12-2011 12:30,9.95,14649.0,United Kingdom,[[ 8.09799694e-03 2.99317725e-02 -4.16102819e...
19996,220867,556237,21001,ROSE DU SUD WASHBAG,1,09-06-2011 15:34,4.13,NaN,United Kingdom,[[ 1.56303719e-02 5.09034134e-02 -1.61667652e...
19997,155012,549958,22208,WOOD STAMP SET THANK YOU,2,13-04-2011 13:47,1.65,17589.0,United Kingdom,[[ 1.60321537e-02 5.47334403e-02 -3.10262684e...
19998,76800,542659,22090,PAPER BUNTING RETROSPOT,6,31-01-2011 12:09,2.95,15738.0,United Kingdom,[[ 3.22430693e-02 2.11675446e-02 -2.77375784e...


In [89]:
import numpy as np

# Create or load your list of lists
list_of_lists = df1.Embeddings

# Convert the strings to floats, ignoring non-numeric values
array_of_arrays = []
for inner_list in list_of_lists:
    numeric_values = [float(value) if value.replace(".", "", 1).isdigit() else np.nan for value in inner_list]
    array_of_arrays.append(numeric_values)

# Convert the list of lists to a NumPy array
numpy_array = np.array(array_of_arrays)

print(numpy_array)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (20000,) + inhomogeneous part.

In [98]:
np.savez("data.npz", **df.to_dict())

# Load the DataFrame from the .npz file
loaded_data = np.load("data.npz")
loaded_df = pd.DataFrame(loaded_data)

In [100]:
import pandas as pd

# Create or load your DataFrame with arrays in the "embeddings" column
# df = ...

# Save the DataFrame to an HDF5 file
df.to_hdf("data.h5", key="df_key", mode="w")

# Load the DataFrame from the HDF5 file
loaded_df = pd.read_hdf("data.h5", key="df_key")

print(loaded_df)


       InvoiceNo StockCode                       Description  Quantity  \
213256    555526     23199                  JUMBO BAG APPLES       100   
409879    572077     21485  RETROSPOT HEART HOT WATER BOTTLE         6   
445700    574856     23284     DOORMAT KEEP CALM AND COME IN         3   
369301    569026     47566                     PARTY BUNTING         4   
77336     542719     21986  PACK OF 12 PINK POLKADOT TISSUES         4   
...          ...       ...                               ...       ...   
525658    580635     22767       TRIPLE PHOTO FRAME CORNICE          2   
220867    556237     21001              ROSE DU SUD WASHBAG          1   
155012    549958     22208          WOOD STAMP SET THANK YOU         2   
76800     542659     22090           PAPER BUNTING RETROSPOT         6   
154450    549911     15036         ASSORTED COLOURS SILK FAN        60   

             InvoiceDate  UnitPrice  CustomerID         Country  \
213256  05-06-2011 11:42       1.79     1436

/var/folders/kr/9nlwnrjj3fl33l2k5m7w4snh0000gp/T/ipykernel_99578/3811121187.py:7: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['InvoiceNo', 'StockCode', 'Description', 'InvoiceDate', 'Country',
       'Embeddings'],
      dtype='object')]

  df.to_hdf("data.h5", key="df_key", mode="w")


In [105]:
loaded_df.Embeddings

213256    [[0.019880796, 0.06821113, -0.03838431, 0.0200...
409879    [[0.030709248, 0.042524725, -0.016905049, 0.00...
445700    [[0.043934163, 0.040514313, -0.036823496, 0.03...
369301    [[0.020661157, 0.047775645, -0.014194804, 0.02...
77336     [[0.005416592, 0.002898387, -0.019814478, 0.01...
                                ...                        
525658    [[0.008097997, 0.029931773, -0.041610282, 0.02...
220867    [[0.015630372, 0.050903413, -0.016166765, 0.01...
155012    [[0.016032154, 0.05473344, -0.031026268, 0.016...
76800     [[0.03224307, 0.021167545, -0.027737578, 0.020...
154450    [[-0.024462353, 0.032406785, -0.017173741, 0.0...
Name: Embeddings, Length: 20000, dtype: object